In [6]:
import tensorflow as tf
import pandas as pd 
import os 

2026-01-18 14:00:03.010500: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-18 14:00:03.045955: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-18 14:00:03.938082: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/home/legramante/Documentos/catandogs/venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [7]:
BASE_DIR = '/home/legramante/Documentos/catandogs/cats_and_dogs'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VALID_DIR = os.path.join(BASE_DIR, 'val')
TEST_DIR = os.path.join(BASE_DIR, 'test')

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

Found 17500 files belonging to 2 classes.


2026-01-18 14:00:07.882850: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    VALID_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

Found 3750 files belonging to 2 classes.


In [10]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=False
)

Found 3750 files belonging to 2 classes.


In [11]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [12]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMAGE_SIZE + (3,),
    include_top=False,
    weights='imagenet'
)

In [13]:
base_model.trainable = False

In [14]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [14]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 386s 701ms/step - accuracy: 0.9754 - loss: 0.0766 - val_accuracy: 0.9845 - val_loss: 0.0462
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 375s 684ms/step - accuracy: 0.9843 - loss: 0.0437 - val_accuracy: 0.9864 - val_loss: 0.0440
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 377s 689ms/step - accuracy: 0.9862 - loss: 0.0383 - val_accuracy: 0.9867 - val_loss: 0.0400
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 382s 697ms/step - accuracy: 0.9877 - loss: 0.0346 - val_accuracy: 0.9875 - val_loss: 0.0394
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 369s 674ms/step - accuracy: 0.9882 - loss: 0.0329 - val_accuracy: 0.9867 - val_loss: 0.0400
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 386s 705ms/step - accuracy: 0.9894 - loss: 0.0301 - val_accuracy: 0.9877 - val_loss: 0.0393
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 365s 668ms/step - accuracy: 0.9898 - loss: 0.0285 - val_accuracy: 0.9864 - val_loss: 0.0401
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 368s 673ms/step - accuracy: 0.9913 -

In [14]:
model.save('/home/legramante/Documentos/catandogs/models/cat_dog_classifier.h5')

In [16]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Acurácia no teste: {test_acc:.4f}")

118/118 ━━━━━━━━━━━━━━━━━━━━ 69s 574ms/step - accuracy: 0.6840 - loss: 0.5999
Acurácia no teste: 0.6840


In [18]:
history_df = pd.DataFrame(history.history)

history_df["experiment"] = "baseline_transfer_learning"
history_df["augmentation"] = "no"
history_df["fine_tuning"] = "no"
history_df["notes"] = "Baseline com MobileNetV2 congelada"

history_df.to_csv("/home/legramante/Documentos/catandogs/experiments/baseline_results.csv", index=False)

In [2]:
import tensorflow as tf 

data_argumentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

2026-01-16 09:39:18.515606: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-16 09:39:19.347316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-16 09:39:20.808240: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
/home/legramante/Documentos/catandogs/venv/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
2026-01-16 09:39:21.484678: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit

In [11]:
train_ds_aug = train_ds.map(
    lambda x, y: (data_argumentation(x, training=True), y)
)

In [17]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [20]:
history_aug = model.fit(
    train_ds_aug,
    validation_data=val_ds,
    epochs=EPOCHS
)

Epoch 1/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 419s 761ms/step - accuracy: 0.9091 - loss: 0.2217 - val_accuracy: 0.9771 - val_loss: 0.0735
Epoch 2/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 409s 748ms/step - accuracy: 0.9326 - loss: 0.1651 - val_accuracy: 0.9773 - val_loss: 0.0673
Epoch 3/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 401s 732ms/step - accuracy: 0.9380 - loss: 0.1570 - val_accuracy: 0.9805 - val_loss: 0.0626
Epoch 4/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 415s 758ms/step - accuracy: 0.9393 - loss: 0.1531 - val_accuracy: 0.9773 - val_loss: 0.0691
Epoch 5/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 411s 751ms/step - accuracy: 0.9403 - loss: 0.1467 - val_accuracy: 0.9784 - val_loss: 0.0623
Epoch 6/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 448s 819ms/step - accuracy: 0.9413 - loss: 0.1442 - val_accuracy: 0.9789 - val_loss: 0.0598
Epoch 7/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 437s 798ms/step - accuracy: 0.9438 - loss: 0.1410 - val_accuracy: 0.9805 - val_loss: 0.0589
Epoch 8/10
547/547 ━━━━━━━━━━━━━━━━━━━━ 434s 792ms/step - accuracy: 0.9433 -

In [16]:
model.save('/home/legramante/Documentos/catandogs/models/cat_dog_classifier2.h5')

In [18]:
test_loss_aug, test_acc_aug = model.evaluate(test_ds)
print(f"Acurácia no teste (Data Augmentation): {test_acc_aug:.4f}")


118/118 ━━━━━━━━━━━━━━━━━━━━ 68s 564ms/step - accuracy: 0.6840 - loss: 0.5999
Acurácia no teste (Data Augmentation): 0.6840


In [22]:
history_aug_df = pd.DataFrame(history_aug.history)

history_aug_df["experiment"] = "data_augmentation"
history_aug_df["augmentation"] = "yes"
history_aug_df["fine_tuning"] = "no"
history_aug_df["notes"] = "RandomFlip, RandomRotation, RandomZoom aplicados no treino"

history_aug_df.to_csv(
    "/home/legramante/Documentos/catandogs/experiments/exp_data_augmentation.csv",
    index=False
)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)